In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Function to generate a wave
def generate_wave(duration, sampling_rate, frequency, wave_type='sinusoidal', amplitude=1.0, offset=0.0):
    t = np.arange(0, duration, 1 / sampling_rate)
    if wave_type == 'sinusoidal':
        return amplitude * np.sin(2 * np.pi * frequency * t) + offset
    elif wave_type == 'square':
        return amplitude * np.sign(np.sin(2 * np.pi * frequency * t)) + offset
    else:
        raise ValueError("Unsupported wave_type. Supported types: 'sinusoidal', 'square'")

# Parameters for the waves
duration = 2  # seconds
sampling_rate = 1000  # samples per second
slow_frequency = 1  # Hz (slow sinusoidal frequency)
fast_frequency = 50  # Hz (fast square wave frequency)
slow_amplitude = 1.0
fast_amplitude = 0.5

# Calculate the minimum value for the combined signal
combined_min = slow_amplitude + fast_amplitude

# Calculate offsets based on the minimum value
slow_offset = combined_min - slow_amplitude
fast_offset = combined_min - fast_amplitude

# Generate the waves with the calculated offsets
slow_sinusoidal_wave = generate_wave(duration, sampling_rate, slow_frequency, 'sinusoidal', slow_amplitude, slow_offset)
fast_square_wave = generate_wave(duration, sampling_rate, fast_frequency, 'square', fast_amplitude, fast_offset)

# Combine the waves
combined_wave = slow_sinusoidal_wave + fast_square_wave

# Time array for plotting
t = np.arange(0, duration, 1 / sampling_rate)

# Plot the individual waves and the combined signal
plt.figure(figsize=(10, 8))
plt.subplot(3, 1, 1)
plt.plot(t, slow_sinusoidal_wave)
plt.title("Slow Sinusoidal Wave")
plt.subplot(3, 1, 2)
plt.plot(t, fast_square_wave)
plt.title("Fast Square Wave")
plt.subplot(3, 1, 3)
plt.plot(t, combined_wave)
plt.title("Combined Signal")
plt.tight_layout()
plt.show()


In [ ]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt

def generate_signal(slow_freq, fast_freq, duration, sampling_rate):
    t = np.linspace(0, duration, int(duration * sampling_rate), endpoint=False)
    signal = np.cos(2 * np.pi * (slow_freq * t + 0.5 * np.sin(2 * np.pi * fast_freq * t)))
    return t, signal

def iq_demodulation(t, signal, fast_freq):
    I = signal * np.cos(2 * np.pi * fast_freq * t)
    Q = signal * np.sin(2 * np.pi * fast_freq * t)
    return I, Q

def unwrap_phase(I, Q):
    phase = np.arctan2(Q, I)
    unwrapped_phase = np.unwrap(phase)
    return unwrapped_phase

def plot_signal_and_phase(t, signal, phase, unwrapped_phase):
    plt.figure(figsize=(10, 8))
    plt.subplot(3, 1, 1)
    plt.plot(t, signal)
    plt.title('Original Signal')
    plt.xlabel('Time (s)')
    plt.ylabel('Amplitude')

    plt.subplot(3, 1, 2)
    plt.plot(t, phase, label='Original Phase')
    plt.title('Original Phase')
    plt.xlabel('Time (s)')
    plt.ylabel('Phase (radians)')
    plt.legend()

    plt.subplot(3, 1, 3)
    plt.plot(t, unwrapped_phase, label='Unwrapped Phase')
    plt.title('Unwrapped Phase')
    plt.xlabel('Time (s)')
    plt.ylabel('Phase (radians)')
    plt.legend()

    plt.tight_layout()
    plt.show()

# Parameters
slow_freq = 10  # Slow frequency component
fast_freq = 500  # Fast frequency component
duration = 10   # Signal duration in seconds
sampling_rate = 50000  # Sampling rate in Hz

# Generate the sinusoidal signal
t, signal = generate_signal(slow_freq, fast_freq, duration, sampling_rate)

# Perform I/Q demodulation
I, Q = iq_demodulation(t, signal, fast_freq)

# Unwrap the phase
unwrapped_phase = unwrap_phase(I, Q)

# Plot the original signal, original phase, and unwrapped phase
plot_signal_and_phase(t, signal, np.angle(I + 1j * Q), unwrapped_phase)


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

def generate_iq_signal(freq, amplitude, phase_offset, num_samples):
    """
    Generate IQ signals from a sinusoidal waveform.

    Parameters:
        freq (float): Frequency of the sinusoidal signal in Hz.
        amplitude (float): Amplitude of the sinusoidal signal.
        phase_offset (float): Phase offset of the sinusoidal signal in radians.
        num_samples (int): Number of samples to generate.

    Returns:
        numpy.array: Complex-valued array representing IQ signals.
    """
    t = np.linspace(0, 1, num_samples)
    iq_signal = amplitude * np.exp(1j * (2 * np.pi * freq * t + phase_offset))
    return t, iq_signal

def unwrap_phase(iq_signal):
    """
    Unwrap the phase of IQ signals.

    Parameters:
        iq_signal (numpy.array): Complex-valued array representing IQ signals.

    Returns:
        numpy.array: Unwrapped phase array in radians.
    """
    unwrapped_phase = np.angle(iq_signal)
    diff = np.diff(unwrapped_phase)

    # Find phase jumps greater than π and correct them
    jumps = np.where(np.abs(diff) > np.pi)[0]
    for jump_idx in jumps:
        if diff[jump_idx] > np.pi:
            unwrapped_phase[jump_idx + 1:] -= 2 * np.pi
        elif diff[jump_idx] < -np.pi:
            unwrapped_phase[jump_idx + 1:] += 2 * np.pi

    return unwrapped_phase

# Parameters for the sinusoidal signal
frequency = 5  # Frequency in Hz
amplitude = 1.0
phase_offset = np.pi / 4  # 45 degrees in radians
num_samples = 1000

# Generate IQ signal
t, iq_signal = generate_iq_signal(frequency, amplitude, phase_offset, num_samples)

# Unwrap the phase
unwrapped_phase = unwrap_phase(iq_signal)

# Plot the original and unwrapped phase
plt.figure()
plt.plot(t, np.real(iq_signal))
plt.grid(True)
plt.show()

plt.figure(figsize=(12, 6))
plt.subplot(2, 1, 1)
plt.plot(np.angle(iq_signal), label='Original Phase')
plt.title('Original Phase of IQ Signal')
plt.xlabel('Sample Index')
plt.ylabel('Phase (radians)')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(unwrapped_phase, label='Unwrapped Phase')
plt.title('Unwrapped Phase of IQ Signal')
plt.xlabel('Sample Index')
plt.ylabel('Unwrapped Phase (radians)')
plt.legend()

plt.tight_layout()
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation

def unwrap_phase(iq_signal):
    """
    Unwrap the phase of IQ signals.
    
    Parameters:
        iq_signal (numpy.array): Complex-valued array representing IQ signals.

    Returns:
        numpy.array: Unwrapped phase array in radians.
    """
    unwrapped_phase = np.angle(iq_signal)
    diff = np.diff(unwrapped_phase)

    # Find phase jumps greater than π and correct them
    jumps = np.where(np.abs(diff) > np.pi)[0]
    for jump_idx in jumps:
        if diff[jump_idx] > np.pi:
            unwrapped_phase[jump_idx + 1:] -= 2 * np.pi
        elif diff[jump_idx] < -np.pi:
            unwrapped_phase[jump_idx + 1:] += 2 * np.pi

    return unwrapped_phase

# Real-time plot initialization
fig, ax = plt.subplots()
ax.set_xlim(0, 2)  # Adjust the x-axis limits as needed
ax.set_ylim(-np.pi, np.pi)  # Adjust the y-axis limits as needed
line, = ax.plot([], [], label='Unwrapped Phase')
ax.legend()

# Real-time phase unwrapping
num_samples = 100
frequency = 5  # Frequency of the sinusoidal signal in Hz
sampling_rate = 100  # Sampling rate in Hz
time = np.arange(num_samples) / sampling_rate

buffer_size = 50  # Circular buffer size
iq_buffer = np.zeros(buffer_size, dtype=np.complex128)
phase_buffer = np.zeros(buffer_size)
current_idx = 0

def update(i):
    global current_idx

    # Generate the new IQ sample and add it to the circular buffer
    amplitude = 1.0
    phase_offset = np.pi / 4
    iq_sample = amplitude * np.exp(1j * (2 * np.pi * frequency * time[current_idx] + phase_offset))
    iq_buffer[current_idx] = iq_sample

    # Perform phase unwrapping on the circular buffer
    unwrapped_phase = unwrap_phase(iq_buffer)

    # Update the phase buffer with the new unwrapped phase value
    phase_buffer[current_idx] = unwrapped_phase[-1]

    # Increment the buffer index in a circular manner
    current_idx = (current_idx + 1) % buffer_size

    # Update the real-time plot
    line.set_data(time[:current_idx], phase_buffer[:current_idx])

    return line,

# Animate the real-time plot
ani = animation.FuncAnimation(fig, update, frames=num_samples, interval=1)
plt.xlabel('Time (seconds)')
plt.ylabel('Phase (radians)')
plt.title('Real-time Phase Unwrapping')
plt.show()

In [ ]:
private double[] PhaseEstimate(double[] bitSeq)
        {
            int j, jm, n, i, im;
            double I, Q, alpha, oldalpha, pi, dt, phi0;
            double[] phi;
            //double[] phidec;
            //double[][] tmp;

            phi = null;
            try
            {
                pi = Math.PI;
                jm = bitSeq.Length; im = jm >> 1;
                phi = new double[im]; time = new double[im];
                phi0 = alpha = Math.Atan(bitSeq[0] / bitSeq[1]); phi[0] = 0;
                n = 0; i = 1; dt = 1.0 / hsFreq;
                while (i < im)
                {
                    //oldalpha = phi[i - 1] - n * pi;
                    oldalpha = alpha;
                    j = 2 * i;
                    I = bitSeq[j];
                    Q = bitSeq[j + 1];
                    alpha = Math.Atan(I / Q);
                    if (oldalpha > 0 && (oldalpha - alpha) > pi / 2) n += 1;
                    if (oldalpha < 0 && (alpha - oldalpha) > pi / 2) n -= 1;
                    phi[i] = alpha + n * pi - phi0;
                    time[i] = i * dt;
                    i += 1;
                }
                bUserEntry = false;
                nudTmin.Maximum = nudTmax.Maximum = (decimal)time[im - 1];
                nudTmin.Value = (decimal)time[0];
                nudTmax.Value = (decimal)time[i - 1];
                PlotPhase(time, phi, time[0], time[im - 1]);

            }
            catch (Exception ex)
            {
                MessageBox.Show(ex.Message);
            }
            return phi;
        }

In [ ]:
def phaseEstimate(data):
    global fastSamplingFreq

    jBuf = len(data); iBuf = j >> 1
    phiInit = np.atan(data[0]/data[1]); alphaInit = phiInit
    n, i, dt = 0, 1, 1/fastSamplingFreq
    phi, time = [], []
    
    phi.append(0)
    while i<iBuf:
        alphaBuf = alpha
        j = 2*i
        inphase = data[j]; quadrature = data[j+1]
        alpha = np.atan(inphase/quadrature)

        if alphaBuf > 0 and (alphaBuf-alpha) > np.pi/2: n += 1
        if alphaBuf < 0 and (alpha-alphaBuf) > np.pi/2: n -= 1

        phi.append(alpha + n*np.pi - phiInit)
        time.append(i*dt)

        i += 1

In [ ]:
private void AnalogInCallback_2(IAsyncResult ar)
        {
            double avg1, avg2;
            double[] aidata;
            int j, i, ij;
            double I, Q, oldalpha, elapsedTime;
            const int jm = 1, jm2 = 3;
            try
            {
                if (runningTask != null && runningTask == ar.AsyncState)
                {
                    // Read the available data from the channels
                    data = analogInReader.EndReadWaveform(ar);
                    aidata = data.GetScaledData();

                    for (i = 0; i < numOfHsCyclesToRead; i++)
                    {
                        avg1 = avg2 = 0;
                        ij = i * hsPointsPerCycle;
                        for (j = -jm; j <= jm; j++)
                        {
                            avg1 += aidata[ij + sampleOffset + j + phaseShifterDelay];
                            avg2 += aidata[ij + sampleOffset2 + j + phaseShifterDelay];
                        }
                        avg1 /= jm2;
                        avg2 /= jm2;
                        I = avg1;
                        Q = avg2;
                        if (counter == 0)
                        {
                            phi = 0;
                            phi0 = alpha = Math.Atan(I / Q);
                            n = 0;
                            startTime = stopwatch.Elapsed.TotalSeconds;
                        }
                        else
                        {
                            //oldalpha = phi - n * Math.PI;
                            oldalpha = alpha;
                            alpha = Math.Atan(I / Q);
                            if (oldalpha > 0 && (oldalpha - alpha) > Math.PI / 2) n += 1;
                            if (oldalpha < 0 && (alpha - oldalpha) > Math.PI / 2) n -= 1;
                            phi = alpha + n * Math.PI - phi0;
                        }

                        if (i == 0)
                        {
                            phimx = phimn = phi;
                            phiav = 0;
                        }
                        counter++;
                        phiav += phi;
                        if (phimx <= phi) phimx = phi;
                        if (phimn >= phi) phimn = phi;
                    }

                    phiav /= numOfHsCyclesToRead;
                    elapsedTime = stopwatch.Elapsed.TotalSeconds - startTime;
                    using (BinaryWriter binWriter = new BinaryWriter(stream, Encoding.UTF8, true))
                    {
                        binWriter.Write(elapsedTime);
                        binWriter.Write(phimx);
                        binWriter.Write(phimn);
                        binWriter.Write(phiav);
                        if (ckbDelayLine.Checked) binWriter.Write(Convert.ToDouble(cumStepChange));
                    }
                    nudDuration.Value = (decimal)elapsedTime;
                    if (ckbDelayLine.Checked)
                    {
                        if (phiav >= maxAllowedPhase)
                        {
                            OzOptComm.StepIncrease(-compSteps);
                            cumStepChange -= compSteps;
                        }
                        if (phiav <= -maxAllowedPhase)
                        {
                            OzOptComm.StepIncrease(compSteps);
                            cumStepChange += compSteps;
                        }
                        tbxStepCumChange.Text = cumStepChange.ToString();
                    }
                }
                analogInReader.BeginMemoryOptimizedReadWaveform(numOfSamplesToRead, analogCallback, aiTask, data);
            }

In [ ]:
; INI file for Voltage Driver


[PC_AO]
;polarization controller
AO_CHNLS=Dev1/ao0,Dev1/ao1,Dev1/ao2
MIN_VOLT=-10
MAX_VOLT=10
DAC_VOLTAGE_INCREMENT=0.02

[PS_AO]
;phase shifter
AO_CHNLS=Dev1/ao2
MIN_VOLT=-10
MAX_VOLT=10



[SINE_WAVE]
; THE PREFIXES SS_ and HS stand for Slow Speed and High Speed recpectively
SS_FREQUENCY=1
SS_CYCLES_PER_BUFFER=1
; AMPLITUDE IS PEAK TO PEAK
HS_FREQUENCY=500
; should be interger* ss frequency; everything after the = is treated as text
HS_POINTS_PER_CYCLE=100
SS_AMPLITUDE=0 
HS_AMPLITUDE=8.75
OFFSET=0 
;more offset for piezo better response

;For the button modulation
PHASE_MOD_FREQUENCY=20 
;frequency is 20, just ignore ""phase mod""
PHASE_MOD_NUM_OF_SAMPLES_PER_CYCLE=200
PHASE_MOD_CYCLES_PER_BUFFER=1
PHASE_MOD_AMPLITUDE=8.75
PHASE_MOD_OFFSET=0



[DAQ_AI]
;AI_CHNL=Dev2/ai3
AI_CHNL=Dev1/ai16
MIN_VOLT=-10
MAX_VOLT=10
;AI_SCAN_RATE=2000
NUM_OF_HS_CYCLES_TO_READ=1000


[AMPLIFIER]
GAIN=30

[PHASE_SHIFTER]
;DELAY_PERCENT=24
DELAY_PERCENT=0

[PLOT]
MAX_NUM_OF_POINTS_TO_PLOT=200000

[OZ_DELAY_LINE]
PORT=COM4
MAX_STEPS=600000
[PHASE_CONTROL]
MAX_ALLOWED_PHASE_RAD=5
PHASE_GUARD=2
COMP_STEPS=2